In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from pinecone import Pinecone,ServerlessSpec
from pypdf import PdfReader

In [ ]:
BASE_DIR = Path(os.getcwd()).parent
PATH_ENV = os.path.join(BASE_DIR,".env")
load_dotenv(override=True,dotenv_path=PATH_ENV)
openai_key_pinecone = os.getenv("token_pinecone")

c:\Users\ASUS\Laboratorio\Landing\terraform-aws-rag\pinecone_rag\.env


In [ ]:
pc  = Pinecone(api_key=openai_key_pinecone)
print(pc.list_indexes())

In [3]:
pdf_file = r"C:\Users\ASUS\Laboratorio\Landing\terraform-aws-rag\pinecone_rag\names_spaces\investigations\Constructing a language for testing RL.pdf"
reader = PdfReader(pdf_file)

In [119]:
len(reader.pages)

75

In [4]:
pdf_texts = [p.extract_text().strip() for p in reader.pages]

In [113]:
pdf_texts

['',
 'Maestría en Ingeniería \nde Sistemas \ny Computación\nDissertation\nConstructing a language for testing\nReinforcement Learning programs using\nNLP techniques\nLUIS ALEJANDRO MEDINA\nJuly 19, 2025\nThis thesis is submitted in partial fulfillment of the requirements\nfor a degree of Master in Systems and Computing Engineering\n(MISIS).\nThesis Committee:\nProf. Nicolás Cardozo (Promotor) Universidad de los Andes, Colombia\nProf. Reviewer 1Ivana Dusparic Trinity College Dublin, Ireland\nProf. Reviewer 2Ruben Manrique Universidad de los Andes, Colombia',
 'Constructing a language for testing Reinforcement Learning pro-\ngrams using NLP techniques\n© YEAR LUIS ALEJANDRO MEDINA\nSystems and Computing Engineering Department\nFLAG lab\nFaculty of Engineering\nUniversidad de los Andes\nBogotá, Colombia',
 'With all my heart: To my dear dad, Luis Hernando Medina,\neverything I have achieved is thanks to you. To all my family,\nthank you for never abandoning me. And to my beloved Martín, 

In [ ]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
)

character_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n","\n",". ", " ", ""],
    chunk_size = 1000, 
    chunk_overlap=0
)

##### split recursive method
character_split_texts = character_splitter.split_text()

In [111]:
len_map_characters = map(lambda x: len(x) , character_split_texts)
max(len_map_characters)

996

In [ ]:
######
## 
######
token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=0,tokens_per_chunk=256
)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

In [108]:
token_split_texts

['maestria en ingenieria de sistemas y computacion dissertation constructing a language for testing reinforcement learning programs using nlp techniques luis alejandro medina july 19, 2025 this thesis is submitted in partial fulfillment of the requirements for a degree of master in systems and computing engineering ( misis ). thesis committee : prof. nicolas cardozo ( promotor ) universidad de los andes, colombia prof. reviewer 1ivana dusparic trinity college dublin, ireland prof. reviewer 2ruben manrique universidad de los andes, colombia constructing a language for testing reinforcement learning pro - grams using nlp techniques © year luis alejandro medina systems and computing engineering department flag lab faculty of engineering universidad de los andes bogota, colombia',
 'with all my heart : to my dear dad, luis hernando medina, everything i have achieved is thanks to you. to all my family, thank you for never abandoning me. and to my beloved martin, i will always love you.... a

In [ ]:
pc.create_index(
    name="tweens",
    dimension =384,
    metric="dotproduct", 
    spec=ServerlessSpec(cloud="aws",region="us-east-1")
)
print(pc.list_indexes())

In [ ]:
idx = pc.Index("tweens")
description = pc.describe_index("tweens")
print(description.status) 

In [92]:
for namespace in idx.list_namespaces():
    print(namespace)

{
    "name": "__default__",
    "record_count": "158"
}


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer("all-MiniLM-L6-v2",device='cpu')
embeddings = list(map(lambda x: model.encode(x).tolist(),token_split_texts))
idx.delete(delete_all=True )
embddings_list = [(str(i),embeddings[i],{"text":token_split_texts[i]}) for i in range(len(embeddings))]
idx.upsert(embddings_list,namespace="Thesis testing RL with NLP techniques")  

{'upserted_count': 158}

In [114]:
query_0 = idx.query(
    vector = [0 for _ in range(384)], 
    top_k = 2, 
    include_values = True, 
    include_metadata=True,
    # filter = {
    #     "topic":{"$eq":"Master thesis"},
    # }    
)

query_about = idx.query(
    vector = model.encode("which is the problem?").tolist(),
    top_k = 100, 
    include_values = True, 
    include_metadata=True, 
    # filter = {
    #     "topic":{"$eq": "Master thesis"}
    # }
)


In [115]:
for result in query_about["matches"]: 
    print(f"score: {result['score']}: {result['metadata']}")

score: 0.249157906: {'text': '................................... 43 6. 2. limitations....................................... 44 6. 3. future work...................................... 45 a. appendix.......................................... 47'}
score: 0.202689171: {'text': '1. introduction 4'}
score: 0.189448357: {'text': 'taking of decisions through the sequence of states. 30'}
score: 0.175964355: {'text': '5. 6. evaluation and results................................. 31 5. 6. 1. cartpole..................................... 31 5. 6. 2. lunar lander.................................. 33 5. 6. 3. a small example................................ 36 5. 6. 4. threats to validity............................... 41 6. conclusion, limitations and future work........................ 43 6. 1. conclusion....'}
score: 0.174908161: {'text': '_ v0. ipynb, 2019. accessed : may 15, 2025.'}
score: 0.170476913: {'text': '. failure probability............................... 12 3. 3. 3. random forest....

In [86]:
ids_query = [result["id"] for result in query_matches_about]
res = idx.fetch(ids=ids_query)

In [135]:
CONTROL_CHARS = ''.join(map(chr, range(0, 32))) + chr(127)


In [140]:
CONTROL_CHARS

'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f\x7f'

In [141]:
import re

re.escape(CONTROL_CHARS.replace('\n','').replace('\t',''))

'\x00\x01\x02\x03\x04\x05\x06\x07\x08\\\x0b\\\x0c\\\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f\x7f'

In [131]:
"hola mundo".replace("h","x")

'xola mundo'

In [138]:
import re

CONTROL_CHARS = ''.join(map(chr, list(range(0,32)) + [127]))

# 👇 OJO: usa '\n' y '\t' (no raw strings)
s = CONTROL_CHARS.replace('\n', '').replace('\t', '')

print('\n in s? ', '\n' in s)  # debe ser False
print('\t in s? ', '\t' in s)  # debe ser False

esc = re.escape(s)
print(repr(esc))



 in s?  False
	 in s?  False
'\x00\x01\x02\x03\x04\x05\x06\x07\x08\\\x0b\\\x0c\\\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f\x7f'
